## 練習時間
將你的結果存成 csv, 上傳你的第一份 Kaggle 成績

Hints: https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file

In [93]:
# Import 需要的套件
import os
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [99]:
# 設定 data_path
dir_data = '../Data/Part01/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

   SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

              ...              FLAG_DOCUMENT_18 FLAG_DOCUMENT_19  \
0             ...                             0 

In [100]:
for col in app_train:
    #Step1 分析各欄位的資料型態
    #print("app_train[{}].dtype={}".format(col,app_train[col].dtype))
    
    #Step2 把各欄位重複的資料濾掉
    #print("app_train[{}].unique={}".format(col,app_train[col].unique()))
    
    #step3 印出資料型態為object欄位名稱
    if app_train[col].dtype == 'object':        
        if len(list(app_train[col].unique())) <= 2:
            #step4 並且印出資料種類只有兩種獲一種的欄位名稱
            print("col={}".format(col))
            
            #step4ㄅㄡ 並且印出資料種類只有兩種獲一種的資料
            #print("app_train[{}]={}".format(col,app_train[col]))
            

            
            

col=NAME_CONTRACT_TYPE
col=FLAG_OWN_CAR
col=FLAG_OWN_REALTY


In [101]:
# Iterate through the columns
for col in app_train:    
    if app_train[col].dtype == 'object':
        #print("col={}".format(col))
        # If 2 or fewer unique categories        
        if len(list(app_train[col].unique())) <= 2:
            
            #  開始針對'col'欄位，進行訓練
            le.fit(app_train[col])
            #print("1 app_train[{}]={}".format(col,app_train[col]))
            
            # 將'col'欄位內的類別，轉為整數(Transform Categories Into Integers)，由0開始遞增
            app_train[col] = le.transform(app_train[col])
            #print("2 app_train[{}]={}".format(col,app_train[col]))
            
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
# 將app_train的所有欄位轉為one-hot coding,當種類超過兩種時,會增加欄位   
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
# 新增'DAYS_EMPLOYED_ANOM'紀錄是否等於365243
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# 將365243替換成365243,inplace = True => 會真正替換
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])


### 做好前處理
開始擬合模型之前，我們要確保 training & testing data 的欄位數量一致，原因是因為 One hot encoding 會製造多的欄位，有些類別出現在 training data 而沒有出現 testing data 中，我們就要把這些多餘的欄位去除

In [102]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

   SK_ID_CURR  TARGET  NAME_CONTRACT_TYPE  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0      100002       1                   0             0                1   
1      100003       0                   0             0                0   
2      100004       0                   1             1                1   
3      100006       0                   0             0                1   
4      100007       0                   0             0                1   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          202500.0    406597.5      24700.5         351000.0   
1             0          270000.0   1293502.5      35698.5        1129500.0   
2             0           67500.0    135000.0       6750.0         135000.0   
3             0          135000.0    312682.5      29686.5         297000.0   
4             0          121500.0    513000.0      21865.5         513000.0   

          ...          WALLSMATERIAL_MODE_Block  WALLSMATERIAL_MODE_

In [105]:
from sklearn.preprocessing import MinMaxScaler, Imputer


# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(labels = ['TARGET'], axis=1)
else:
    train = app_train.copy()
    

    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 240)
Testing data shape:  (48744, 240)


In [108]:
print(train[0:1])

[[0.         0.         0.         1.         0.         0.00151187
  0.09028652 0.09003159 0.07744108 0.25632114 0.11116122 0.96443725
  0.85214008 0.70543282 0.0989011  1.         1.         0.
  1.         1.         0.         0.         0.5        0.5
  0.43478261 0.         0.         0.         0.         0.
  0.         0.07221502 0.30754227 0.15505445 0.0247     0.0369
  0.9722     0.6192     0.0143     0.         0.069      0.0833
  0.125      0.0369     0.0202     0.019      0.         0.
  0.0252     0.0383     0.9722     0.6341     0.0144     0.
  0.069      0.0833     0.125      0.0377     0.022      0.0198
  0.         0.         0.025      0.0369     0.9722     0.6243
  0.0144     0.         0.069      0.0833     0.125      0.0375
  0.0205     0.0193     0.         0.         0.0149     0.00574713
  0.05882353 0.00581395 0.08333333 0.73578751 0.         1.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.       

### Fit the model

In [109]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

模型 fit 好以後，就可以用來預測 testing data 中的客戶違約遲繳貸款的機率咯! (記得要用 predict_proba 才會輸出機率)

In [110]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

### 儲存預測結果

In [111]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

d:\ProgramFiles_D\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.065051
1,100005,0.126401
2,100013,0.081239
3,100028,0.061509
4,100038,0.128308


In [112]:
submit.to_csv("submit_J_01.csv", encoding='utf_8', index=False)